In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data\preprocessed.csv")

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\USER\AppData\Local\Temp\ipykernel_4708\222109947.py:1: SyntaxWarning: invalid escape sequence '\p'
  df = pd.read_csv("data\preprocessed.csv")


In [3]:
df.columns

Index(['raw_id', 'clinic_application_date', 'intake_date', 'group',
       'signing_date', 'sheet', 'clean_id', 'therapy_start_date',
       'therapy_end_date'],
      dtype='object')

In [3]:
priority = {
        'סיימו טיפול': 0,
        'CAU': 1,
        'IPC-SSC': 2,
        'משתתפים פעילים': 3,
        'פספוסי גיוסים': 4,
        'אין שת"פ טיפולי': 5,
        'אי התאמה למחקר': 6,
        'אי הסכמה למחקר': 7,
        'נשירה מחקרית': 8,
        'נשירה קלינית- לאחר ת. טיפול': 9,
        'עלייה לרמה 2': 10
}

In [4]:
formats = [
    "%Y-%m-%d",   # 2025-09-28
    "%d/%m/%Y",   # 28/09/2025
    "%m/%d/%Y",   # 09/28/2025
    "%d-%b-%Y",   # 28-Sep-2025
    "%d.%m.%Y",   # 28.09.2025
    '%Y-%m-%d %H:%M:%S'
]

date_columns = [i for i in df.columns if 'date' in i]


In [5]:
def parse_date(x):
    for fmt in formats:
        try:
            return pd.to_datetime(x, format=fmt)
        except (ValueError, TypeError):
            continue
    return pd.NaT

for date_col in date_columns:
    df[date_col] = df[date_col].apply(parse_date)

#### Filling missing data by prioraty 

In [6]:
# map priority and sort so preferred sources come first for each participant
df['prio'] = df['sheet'].map(priority)
df_sorted = df.sort_values(['clean_id', 'prio'])

# group by participant and take the first non-null value per column
# groupby.first() skips NA and returns first non-NA per column
result = df_sorted.groupby('clean_id', as_index=False).first()

# drop helper column if you like
result = result.drop(columns=['prio'])

In [7]:
dummies = pd.get_dummies(df['sheet'], prefix='has_source')
dummies['clean_id'] = df['clean_id']
one_hot_g = dummies.groupby('clean_id', as_index=True).max().reset_index()

# merge similarly
final2 = result.merge(one_hot_g, on='clean_id', how='left').fillna(0)

In [8]:
final2['group'] = final2.group.str.replace('Stepped care', 'Stepped Care')
final2["first_contact_date"] = final2["clinic_application_date"].fillna(final2["intake_date"]).fillna(final2["signing_date"])
final2['therapy_starting_date'] =  final2["therapy_start_date"].fillna(final2["therapy_end_date"])

In [9]:
final2.to_csv(r"data\processed_df_for_consort.csv", index=0)

In [10]:
final2['group'].unique()

array(['CAU', nan, 'Stepped Care', 'טיפול בקבוצת הורים '], dtype=object)

In [11]:
final2.columns

Index(['clean_id', 'raw_id', 'clinic_application_date', 'intake_date', 'group',
       'signing_date', 'sheet', 'therapy_start_date', 'therapy_end_date',
       'has_source_CAU', 'has_source_IPC-SSC', 'has_source_אי הסכמה למחקר',
       'has_source_אי התאמה למחקר', 'has_source_אין שת"פ טיפולי',
       'has_source_משתתפים פעילים', 'has_source_נשירה מחקרית',
       'has_source_נשירה קלינית- לאחר ת. טיפול', 'has_source_סיימו טיפול',
       'has_source_עלייה לרמה 2', 'has_source_פספוסי גיוסים',
       'first_contact_date', 'therapy_starting_date'],
      dtype='object')